In [50]:
import os
import sys

def add_sys_path(p):
    p = os.path.abspath(p)
    print(p)
    if p not in sys.path:
        sys.path.append(p)

In [51]:
toxic_data_path = 'detox/emnlp2021/data/test/test_10k_toxic'

In [52]:
import torch
import numpy as np

In [53]:
add_sys_path('detox/emnlp2021/style_transfer/paraGeDi')

/u1/mdr614/FSE_majorRevision/detox-20240127T202801Z-001/detox/emnlp2021/style_transfer/paraGeDi


In [54]:
from importlib import reload
import gedi_adapter
reload(gedi_adapter)
from gedi_adapter import GediAdapter

In [55]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoTokenizer
t5name = 's-nlp/t5-paraphrase-paws-msrp-opinosis-paranmt'
model_path = 's-nlp/gpt2-base-gedi-detoxification'
clf_name = 's-nlp/roberta_toxicity_classifier_v1'

In [56]:
import text_processing
reload(text_processing);

In [57]:
import torch
#device = torch.device('cuda:0')
device = torch.device('cpu')

In [58]:
tokenizer = AutoTokenizer.from_pretrained(t5name)
para = AutoModelForSeq2SeqLM.from_pretrained(t5name)
para.resize_token_embeddings(len(tokenizer))
para.to(device);
para.eval();

In [59]:
gedi_dis = AutoModelForCausalLM.from_pretrained(model_path)

Some weights of the model checkpoint at s-nlp/gpt2-base-gedi-detoxification were not used when initializing GPT2LMHeadModel: ['bias', 'logit_scale']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
NEW_POS = tokenizer.encode('normal', add_special_tokens=False)[0]
NEW_NEG = tokenizer.encode('toxic', add_special_tokens=False)[0]

In [61]:
# add gedi-specific parameters
if os.path.exists(model_path):
    w = torch.load(model_path + '/pytorch_model.bin', map_location='cpu')
    gedi_dis.bias = w['bias']
    gedi_dis.logit_scale = w['logit_scale']
    del w
else:
    gedi_dis.bias = torch.tensor([[ 0.08441592, -0.08441573]])
    gedi_dis.logit_scale = torch.tensor([[1.2701858]])
print(gedi_dis.bias, gedi_dis.logit_scale)

tensor([[ 0.0844, -0.0844]]) tensor([[1.2702]])


In [62]:
dadapter = GediAdapter(model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=5, target=1, neg_code=NEW_NEG, pos_code=NEW_POS, lb=None, ub=None)
text = 'The internal policy of Trump is flawed.'
print('====BEFORE====')
print(text)
print('====AFTER=====')
inputs = tokenizer.encode(text, return_tensors='pt').to(para.device)
result = dadapter.generate(inputs, do_sample=True, num_return_sequences=3, temperature=1.0, repetition_penalty=3.0, num_beams=1)
for r in result:
    print(tokenizer.decode(r, skip_special_tokens=True))

====BEFORE====
The internal policy of Trump is flawed.
====AFTER=====
the trump has an ugly internal policy.
the president's current and existent domestic policy is corrupt
he is a fool.


In [63]:
import gc

def cleanup():
    gc.collect()
    if torch.cuda.is_available() and device.type != 'cpu':
        with torch.cuda.device(device):
            torch.cuda.empty_cache()

In [64]:
gedi_dis.to(device);
gedi_dis.bias = gedi_dis.bias.to(device)
gedi_dis.logit_scale = gedi_dis.logit_scale.to(device)
gedi_dis.eval();

In [65]:
import pandas as pd

# Specify the path to your CSV file
csv_path = '/u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Random_2000_uncivil_comments_for_conversion/civilAlternatives_by_random2000.csv'  # Replace with your actual CSV file path

# Read the CSV file using pandas
df = pd.read_csv(csv_path)

# Access the 'uncivil' column and convert it to a list
test_data = df['uncivil'].tolist()

# Print the number of sentences
print(len(test_data))

1999


In [66]:
dadapter = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer, gedi_logit_coef=10, target=0, neg_code=NEW_NEG, pos_code=NEW_POS,
    reg_alpha=3e-5, ub=0.01
)

In [67]:
def paraphrase(text, n=None, max_length='auto', beams=2):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(dadapter.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    result = dadapter.generate(
        inputs,
        num_return_sequences=n or 1,
        do_sample=False, temperature=0.0, repetition_penalty=3.0, max_length=max_length,
        bad_words_ids=[[2]],  # unk
        num_beams=beams,
    )
    texts = [tokenizer.decode(r, skip_special_tokens=True) for r in result]
    texts = [text_processing.text_postprocess(t) for t in texts]
    if not n and isinstance(text, str):
        return texts[0]
    return texts

In [68]:
paraphrase(test_data[:3])

['eapBYteBuffer[pos=0 lim=0 cap=0]] output: ByteBufferWrapper (884587465)',
 "it looks like this will result in a fork. That's a real shame, but that's what happens if you throw the community out of the boat.",
 'The lack of consistency in my own ecosystem, however, is a nightmare.']

In [69]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer
clf = RobertaForSequenceClassification.from_pretrained(clf_name).to(device);
clf_tokenizer = RobertaTokenizer.from_pretrained(clf_name)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier_v1 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [70]:
def predict_toxicity(texts):
    with torch.inference_mode():
        inputs = clf_tokenizer(texts, return_tensors='pt', padding=True).to(clf.device)
        out = torch.softmax(clf(**inputs).logits, -1)[:, 1].cpu().numpy()
    return out

In [71]:
from gedi_adapter import GediAdapter

adapter2 = GediAdapter(
    model=para, gedi_model=gedi_dis, tokenizer=tokenizer,
    gedi_logit_coef=10,
    target=0, pos_code=NEW_POS,
    neg_code=NEW_NEG,
    reg_alpha=3e-5,
    ub=0.01,
    untouchable_tokens=[0, 1],
)


def paraphrase(text, max_length='auto', beams=5, rerank=True):
    texts = [text] if isinstance(text, str) else text
    texts = [text_processing.text_preprocess(t) for t in texts]
    inputs = tokenizer(texts, return_tensors='pt', padding=True)['input_ids'].to(adapter2.device)
    if max_length == 'auto':
        max_length = min(int(inputs.shape[1] * 1.1) + 4, 64)
    attempts = beams
    out = adapter2.generate(
        inputs,
        num_beams=beams,
        num_return_sequences=attempts,
        do_sample=False,
        temperature=1.0,
        repetition_penalty=3.0,
        max_length=max_length,
        bad_words_ids=[[2]],  # unk
        output_scores=True,
        return_dict_in_generate=True,
    )
    results = [tokenizer.decode(r, skip_special_tokens=True) for r in out.sequences]

    if rerank:
        scores = predict_toxicity(results)

    results = [text_processing.text_postprocess(t) for t in results]
    out_texts = []
    for i in range(len(texts)):
        if rerank:
            idx = scores[(i*attempts):((i+1)*attempts)].argmin()
        else:
            idx = 0
        out_texts.append(results[i*attempts+idx])
    return out_texts

torch.manual_seed(0)
paraphrase(['fuck you!', 'you are stupid!', 'you remind me of the chump .', 'he has to be a terrorist ! .'], beams=3)

["Fick 'Emmy.",
 "You'd be wrong!",
 "You'll remind me of chump?",
 'Must be a Terrorist!']

In [72]:
batch_size = 2

In [74]:
import os
import csv
from tqdm.auto import tqdm, trange

cleanup()

lines = test_data
output_csv_path = '/u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Random_2000_uncivil_comments_for_conversion/civilAlternatives_by_Paragedi_random2000.csv'

# Open the CSV file in append mode
with open(output_csv_path, 'a', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)

    for i in trange(int(len(lines) / batch_size + 1)):
        if i % 10 == 0:
            cleanup()
        t = i * batch_size
        batch = [line.strip() for line in lines[t:(t+batch_size)]]
        if not batch:
            continue

        try:
            res = paraphrase(batch, max_length='auto', beams=10)
        except RuntimeError:
            print('runtime error for batch ', i)
            try:
                cleanup()
                res = [paraphrase([text], max_length='auto', beams=3)[0] for text in batch]
            except RuntimeError:
                print('runtime error for batch ', i, 'even with batch size 1')
                res = batch
                cleanup()

        for original, out in zip(batch, res):
            # Write the original and paraphrased sentences to the CSV file
            csv_writer.writerow([original, out])

print(f"Results saved in {output_csv_path}")


  0%|          | 0/1000 [00:00<?, ?it/s]

Results saved in /u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Random_2000_uncivil_comments_for_conversion/civilAlternatives_by_Paragedi_random2000.csv
